# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

Date      id Warehouse       Shipments             Customer  \
0  Jun 12 2023  9:42AM  264538        20  ADV80020703-OH  AdvaGen Pharma, Ltd   
1  Jun 12 2023  9:33AM  264537        10        HH9607.1            Hush Hush   
2  Jun 12 2023  9:31AM  264536        10        HH-42530            Hush Hush   
3  Jun 12 2023  9:31AM  264536        10        HH-42547            Hush Hush   
4  Jun 12 2023  9:31AM  264536        10        HH-42548            Hush Hush   
5  Jun 12 2023  9:30AM  264535        18     SEQ-9565677              Seqirus   
6  Jun 12 2023  9:29AM  264534        10        HH-42516            Hush Hush   
7  Jun 12 2023  9:29AM  264534        10        HH-42517            Hush Hush   
8  Jun 12 2023  9:29AM  264534        10        HH-42518            Hush Hush   
9  Jun 12 2023  9:29AM  264534        10        HH-42519            Hush Hush   

  ShipmentStatus  
0       Released  
1       Released  
2       Released  
3       Released  
4       Released  
5       Released  
6       Released  
7       Released  
8       Released  
9       Released

In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

id ShipmentStatus  Shipments
14  264534       Released         43
15  264535       Released          1
16  264536       Released          3
17  264537       Released          1
18  264538       Released          1

In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus  Executing  Released
id                                 
264534                NaN      43.0
264535                NaN       1.0
264536                NaN       3.0
264537                NaN       1.0
264538                NaN       1.0

In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus  Executing  Released
id                                 
264516                0.0       1.0
264517                1.0       0.0
264523                3.0      16.0
264524                2.0       2.0
264525                0.0       1.0

In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus  Executing  Released
id                                 
264516                  0         1
264517                  1         0
264523                  3        16
264524                  2         2
264525                  0         1

In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus      id  Executing  Released
0               264516          0         1
1               264517          1         0
2               264523          3        16
3               264524          2         2
4               264525          0         1

In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus      id Executing Released
0               264516                  1
1               264517         1         
2               264523         3       16
3               264524         2        2
4               264525                  1

In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

Date      id Warehouse                      Customer
0   Jun 12 2023  9:42AM  264538        20           AdvaGen Pharma, Ltd
1   Jun 12 2023  9:33AM  264537        10                     Hush Hush
2   Jun 12 2023  9:31AM  264536        10                     Hush Hush
5   Jun 12 2023  9:30AM  264535        18                       Seqirus
6   Jun 12 2023  9:29AM  264534        10                     Hush Hush
49  Jun 12 2023  9:19AM  264533        19  ThermoFisher Scientific Inc.
50  Jun 12 2023  9:12AM  264532        10                     Hush Hush
51  Jun 12 2023  9:09AM  264531        12              SD Head USA, LLC
52  Jun 12 2023  9:02AM  264529        18          Emersa Waterbox, LLC
53  Jun 12 2023  9:02AM  264530        10           Digital Brands, LLC

In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

Date      id Warehouse                      Customer  \
0  Jun 12 2023  9:42AM  264538        20           AdvaGen Pharma, Ltd   
1  Jun 12 2023  9:33AM  264537        10                     Hush Hush   
2  Jun 12 2023  9:31AM  264536        10                     Hush Hush   
3  Jun 12 2023  9:30AM  264535        18                       Seqirus   
4  Jun 12 2023  9:29AM  264534        10                     Hush Hush   
5  Jun 12 2023  9:19AM  264533        19  ThermoFisher Scientific Inc.   
6  Jun 12 2023  9:12AM  264532        10                     Hush Hush   
7  Jun 12 2023  9:09AM  264531        12              SD Head USA, LLC   
8  Jun 12 2023  9:02AM  264529        18          Emersa Waterbox, LLC   
9  Jun 12 2023  9:02AM  264530        10           Digital Brands, LLC   

  Executing Released  
0                  1  
1                  1  
2                  3  
3                  1  
4                 43  
5         1           
6         1           
7         1           
8                  1  
9                 20

In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

Date      id Warehouse                      Customer  \
0  Jun 12 2023  9:42AM  264538        20           AdvaGen Pharma, Ltd   
1  Jun 12 2023  9:33AM  264537        10                     Hush Hush   
2  Jun 12 2023  9:31AM  264536        10                     Hush Hush   
3  Jun 12 2023  9:30AM  264535        18                       Seqirus   
4  Jun 12 2023  9:29AM  264534        10                     Hush Hush   
5  Jun 12 2023  9:19AM  264533        19  ThermoFisher Scientific Inc.   
6  Jun 12 2023  9:12AM  264532        10                     Hush Hush   
7  Jun 12 2023  9:09AM  264531        12              SD Head USA, LLC   
8  Jun 12 2023  9:02AM  264529        18          Emersa Waterbox, LLC   
9  Jun 12 2023  9:02AM  264530        10           Digital Brands, LLC   

  Released Executing  
0        1            
1        1            
2        3            
3        1            
4       43            
5                  1  
6                  1  
7                  1  
8        1            
9       20

In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

Date      id Warehouse             Customer Released  \
0  Jun 12 2023  9:42AM  264538        20  AdvaGen Pharma, Ltd        1   
1  Jun 12 2023  9:33AM  264537        10            Hush Hush        1   
2  Jun 12 2023  9:31AM  264536        10            Hush Hush        3   
3  Jun 12 2023  9:30AM  264535        18              Seqirus        1   
4  Jun 12 2023  9:29AM  264534        10            Hush Hush       43   

  Executing  
0            
1            
2            
3            
4

In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


Date      id Warehouse             Customer  Released  \
0  Jun 12 2023  9:42AM  264538        20  AdvaGen Pharma, Ltd       1.0   
1  Jun 12 2023  9:33AM  264537        10            Hush Hush       1.0   
2  Jun 12 2023  9:31AM  264536        10            Hush Hush       3.0   
3  Jun 12 2023  9:30AM  264535        18              Seqirus       1.0   
4  Jun 12 2023  9:29AM  264534        10            Hush Hush      43.0   

   Executing  
0        NaN  
1        NaN  
2        NaN  
3        NaN  
4        NaN

In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

Date      id Warehouse             Customer  Released  \
0  Jun 12 2023  9:42AM  264538        20  AdvaGen Pharma, Ltd       1.0   
1  Jun 12 2023  9:33AM  264537        10            Hush Hush       1.0   
2  Jun 12 2023  9:31AM  264536        10            Hush Hush       3.0   
3  Jun 12 2023  9:30AM  264535        18              Seqirus       1.0   
4  Jun 12 2023  9:29AM  264534        10            Hush Hush      43.0   

   Executing  
0        0.0  
1        0.0  
2        0.0  
3        0.0  
4        0.0

In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

id  Released  Executing
Warehouse                              
10         2116244      86.0        6.0
12          264531       0.0        1.0
18         1058116       4.0        0.0
19          529050       0.0        2.0
20          264538       1.0        0.0
22          264516       1.0        0.0

In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

Warehouse       id  Released  Executing
0        10  2116244      86.0        6.0
1        12   264531       0.0        1.0
2        18  1058116       4.0        0.0
3        19   529050       0.0        2.0
4        20   264538       1.0        0.0
5        22   264516       1.0        0.0

In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

Warehouse  Released  Executing
0        10      86.0        6.0
1        12       0.0        1.0
2        18       4.0        0.0
3        19       0.0        2.0
4        20       1.0        0.0
5        22       1.0        0.0

In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

Warehouse    Status  Value
0        10  Released   86.0
1        12  Released    0.0
2        18  Released    4.0
3        19  Released    0.0
4        20  Released    1.0

In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse    10   12   18   19   20   22
Status                                  
Executing   6.0  1.0  0.0  2.0  0.0  0.0
Released   86.0  0.0  4.0  0.0  1.0  1.0

In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse     Status    10   12   18   19   20   22
0          Executing   6.0  1.0  0.0  2.0  0.0  0.0
1           Released  86.0  0.0  4.0  0.0  1.0  1.0

In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

Status    10   12   18   19   20   22
0  Executing   6.0  1.0  0.0  2.0  0.0  0.0
1   Released  86.0  0.0  4.0  0.0  1.0  1.0

In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()